In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import math

## Problem 1

In [3]:
def print_matrix(name, A):
    print(f'Matrix {name}:')
    for i in range(A.shape[0]):
        row = A[i, :]
        print(f'{row[0]:.1f}', end='')
        for j in range(1, row.shape[0]):
            print(f', {row[j]:.1f}', end='')
        print()

In [6]:
def p1_camera(params, points):
    # Extract and convert camera parameters
    rx, ry, rz = [np.radians(deg) for deg in params[0]]  # Convert rotation angles from degrees to radians
    tx, ty, tz = params[1]  # Translation vector
    f, d, ic, jc = params[2]    # Intrinsic parameters
    d /= 1000   # Convert pixel size from microns to millimeters for unit consistency

    # Compute rotation matrices for each axis
    Rx = np.array([[1, 0, 0], [0, np.cos(rx), -np.sin(rx)], [0, np.sin(rx), np.cos(rx)]])
    Ry = np.array([[np.cos(ry), 0, np.sin(ry)], [0, 1, 0], [-np.sin(ry), 0, np.cos(ry)]])
    Rz = np.array([[np.cos(rz), -np.sin(rz), 0], [np.sin(rz), np.cos(rz), 0], [0, 0, 1]])
    
    # Compose the full rotation matrix from individual axes rotations
    R = Rx @ Ry @ Rz
    # Form the intrinsic parameter matrix K
    K = np.array([[f/d, 0, jc], [0, f/d, ic], [0, 0, 1]])
    # Create the translation vector t
    t = np.array([[tx], [ty], [tz]])
    # Calculate the camera matrix M
    M = K @ (np.concatenate((np.transpose(R), -np.transpose(R) @ t), axis=1))
    
    print_matrix("R", R)
    print_matrix("K", K)
    print_matrix("M", M)

    # Determine the optical direction from 3rd col the of rotation matrix
    optical_direction = R[:, 2]
    # Initialize lists to track visibility
    hidden_points = []
    visible_points = []

    print("Projections:")
    # Loop through points to calculate projections, detirmine visibility, and presence in the image
    for num, point in enumerate(points):
        x,y,z = point   # Extract 3D coordinates from point

        # Project the 3D point to 2D using the camera matrix
        up, vp, wp = M @ np.array([x,y,z,1])
        u, v = up / wp, vp / wp # Normalize to obtain image plane coordinates          

        # Determine visibility by checking if the point is in front of the camera plane.
        # This is done by calculating the dot product between the camera's optical direction 
        # vector and the vector from the camera to the point. A positive result indicates the point is 
        # in front of the camera and thus visible. 
        if np.dot(optical_direction, np.array([[x],[y],[z]]) - t) > 0:
            visible_points.append(str(num))
        else:
            hidden_points.append(str(num))
 
        # Check if the projected point falls within the image boundaries
        in_or_out = "inside" if 0 <= v <= 4000 and 0 <= u <= 6000 else "outside"
        print(f"{str(num)}: point {x:.1f} {y:.1f} {z:.1f} ==> {v:.1f}, {u:.1f}, {in_or_out}")

    print(f"visible: {' '.join(visible_points)}")
    print(f"hidden: {' '.join(hidden_points)}")

In [7]:
"""
Problem 1, Test 1. This is a very simple test that could be easily checked by hand.
"""
params = [
    [0.0, 0.0, 0.0], 
    [0.0, 0.0, 10.0],
    [15, 10, 2001, 2995]
]
points = [
    [10, 5, 100],
    [0, 0, 0.5],
    [-30, 10, -20],
    [20, 15, 20]
]

p1_camera(params, points)


Matrix R:
1.0, 0.0, 0.0
0.0, 1.0, 0.0
0.0, 0.0, 1.0
Matrix K:
1500.0, 0.0, 2995.0
0.0, 1500.0, 2001.0
0.0, 0.0, 1.0
Matrix M:
1500.0, 0.0, 2995.0, -29950.0
0.0, 1500.0, 2001.0, -20010.0
0.0, 0.0, 1.0, -10.0
Projections:
0: point 10.0 5.0 100.0 ==> 2084.3, 3161.7, inside
1: point 0.0 0.0 0.5 ==> 2001.0, 2995.0, inside
2: point -30.0 10.0 -20.0 ==> 1501.0, 4495.0, inside
3: point 20.0 15.0 20.0 ==> 4251.0, 5995.0, outside
visible: 0 3
hidden: 1 2


In [8]:
'''
Problem 1, Test 2
'''
params = [
    [15.0, -45.0, 10.0],
    [4.0, 30.0, 10.0],
    [12, 12, 1998, 3005]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]
]
p1_camera(params, points)


Matrix R:
0.7, -0.1, -0.7
-0.0, 1.0, -0.2
0.7, 0.1, 0.7
Matrix K:
1000.0, 0.0, 3005.0
0.0, 1000.0, 1998.0
0.0, 0.0, 1.0
Matrix M:
-1428.5, -562.5, 2770.0, -5112.7
-1535.6, 617.4, 1500.9, -27388.2
-0.7, -0.2, 0.7, 1.5
Projections:
0: point 100.0 15.0 90.0 ==> 3487.2, -8851.4, outside
1: point -100.0 800.0 1500.0 ==> 3021.6, 4043.8, inside
2: point 10.0 -500.0 -500.0 ==> 4311.3, 4394.6, outside
3: point -30.0 10.0 20.0 ==> 1589.0, 2534.4, inside
visible: 1 3
hidden: 0 2


In [9]:
'''
Problem 1, Test 3
'''
params = [
    [-16.0, 10.0, 50.0],
    [25, -12, 50],
    [9, 9, 1000, 1400]
]
points = [
    [100, 15, 90],
    [-100, 800, 1500],
    [10, -500, -500],
    [-30, 10, 20]    
]
p1_camera(params, points)

Matrix R:
0.6, -0.8, 0.2
0.7, 0.7, 0.3
-0.3, -0.0, 0.9
Matrix K:
1000.0, 0.0, 1400.0
0.0, 1000.0, 1000.0
0.0, 0.0, 1.0
Matrix M:
876.1, 1085.6, 1006.9, -59219.4
-580.8, 926.0, 897.4, -19236.6
0.2, 0.3, 0.9, -48.4
Projections:
0: point 100.0 15.0 90.0 ==> 297.8, 2323.9, inside
1: point -100.0 800.0 1500.0 ==> 1352.8, 1420.4, inside
2: point 10.0 -500.0 -500.0 ==> 1428.5, 1672.5, inside
3: point -30.0 10.0 20.0 ==> -794.1, 1704.6, outside
visible: 0 1
hidden: 2 3


## Problem 2

In [10]:
def load_points(fn):
    '''
    Input: a path to a file containing x, y points, one point per line.
    Returns: two-d np array where each row contains an x, y point
    '''
    f = open(fn, 'r')
    pts = []
    for line in f:
        line = line.strip().split()
        x, y = float(line[0]), float(line[1])
        pts.append([x, y])
    pts = np.array(pts)
    f.close()
    return pts

In [26]:
def p2_ransac(fn, samples, tau, seed):
    
    # Load data from file
    with open(fn, 'r') as f:
        data = np.genfromtxt(f, delimiter=' ')

    N = len(data)
    np.random.seed(seed)

    best_inliers,  best_outliers = [], []

    for sample_num in range(samples):
        # Randomly select two points to define a line
        sample = np.random.randint(0, N, 2)
        x1, y1 = data[sample[0]]
        x2, y2 = data[sample[1]]

        # Check if the 2 points are equal, continue
        if sample[0] == sample[1]:
            continue

        # Calculate normalized line equation coefficients:
        a, b, c= -(y2 - y1), (x2-x1), (x1*y2 - x2*y1)
        norm_factor = np.sqrt(a**2+b**2)
        a_norm, b_norm, c_norm = a / norm_factor, b / norm_factor, c / norm_factor

        # calculate inliers by finding distance of each point from line, then comparing to tau
        dists = np.abs(a_norm*data[:, 0] + b_norm*data[:, 1] + c_norm) / np.sqrt(a_norm**2 + b_norm**2)
        inlier_dists = dists[dists < tau]
        outlier_dists = dists[dists >= tau]
   
        # Update and print values if better line is found
        if len(inlier_dists) > len(best_inliers):
            best_inliers, best_outliers = inlier_dists, outlier_dists
            print(f"Sample {sample_num}:")
            print(f"indices ({sample[0]},{sample[1]})")
            print(f"line ({a_norm:.3f},{b_norm:.3f},{c_norm:.3f})")
            print(f"inliers {len(inlier_dists)}\n")

    print(f"avg inlier dist {np.average(best_inliers):.3f}")
    print(f"avg outlier dist {np.average(best_outliers):.3f}")
 

In [27]:
'''
Problem 2, Test 1
'''
fn = 'data/p2_pts1_in.txt'
samples = 25
tau = 2.5
seed = 999
p2_ransac(fn, samples, tau, seed)


Sample 0:
indices (0,28)
line (-0.983,0.184,-26.286)
inliers 13

Sample 3:
indices (27,25)
line (0.426,0.905,-4.913)
inliers 19

Sample 10:
indices (23,4)
line (0.545,0.838,-0.944)
inliers 21

avg inlier dist 0.739
avg outlier dist 8.920


In [28]:
'''
Problem 2, Test 2
'''
fn = 'data/p2_pts2_in.txt'
samples = 35
tau = 3.0
seed = 1232
p2_ransac(fn, samples, tau, seed)


Sample 0:
indices (6,15)
line (0.023,1.000,19.478)
inliers 16

Sample 2:
indices (46,20)
line (-0.178,0.984,11.491)
inliers 21

Sample 4:
indices (75,52)
line (0.500,0.866,-0.018)
inliers 30

Sample 17:
indices (58,18)
line (-0.408,-0.913,-1.201)
inliers 35

avg inlier dist 1.383
avg outlier dist 10.267


## Problem 3 (4270 Only)

Students in 6270 should delete Problem 3 cells from their notebooks prior to submission.

In [11]:
'''
Utility for Problem 3
'''
import os

def get_images(img_dir):
    start_cwd = os.getcwd()
    os.chdir(img_dir)
    img_name_list = os.listdir('./')
    img_name_list = [name for name in img_name_list if 'jpg' in name.lower()]
    img_name_list.sort()

    img_list = []
    for i_name in img_name_list:
        im = cv2.imread(i_name, cv2.IMREAD_GRAYSCALE)
        if im is None:
            print('Could not open', i_name)
            sys.exit(0)
        img_list.append(im)

    os.chdir(start_cwd)
    return img_name_list, img_list

In [47]:
def p3_best_focus(image_dir):
    img_name_list, img_list = get_images(image_dir)
    focus_dict = {}

    for i, im in enumerate(img_list):
        m,n = im.shape

        # Calculate horizontal and vertical gradients using Sobel 
        im_dx = cv2.Sobel(im, cv2.CV_32F, 1, 0)
        im_dy = cv2.Sobel(im, cv2.CV_32F, 0, 1)
  
        # Compute the squared gradient magnitude and normalize by the image area
        sq_gradient_mag = (1 / (m*n)) * np.sum(np.square(im_dx) + np.square(im_dy))

        # Store the computed value: image name in a dictionary
        focus_dict[img_name_list[i]] = sq_gradient_mag

        # Print the squared gradient magnitude for each image
        print(f"{img_name_list[i]}: {sq_gradient_mag:.1f}")  

    # Identify and print the name of the best-focused image
    print(f"Image {max(focus_dict, key=focus_dict.get)} is best focused.")  

In [48]:
image_dir = 'data/evergreen'
p3_best_focus(image_dir)

DSC_1696.JPG: 283.9
DSC_1697.JPG: 312.7
DSC_1698.JPG: 602.4
DSC_1699.JPG: 2137.2
DSC_1700.JPG: 10224.8
DSC_1701.JPG: 18987.1
Image DSC_1701.JPG is best focused.


In [49]:
image_dir = 'data/branches'
p3_best_focus(image_dir)

DSC_1702.JPG: 132.4
DSC_1703.JPG: 1152.0
DSC_1704.JPG: 8229.7
DSC_1705.JPG: 41206.5
DSC_1706.JPG: 22214.3
DSC_1707.JPG: 7876.5
Image DSC_1705.JPG is best focused.
